In [1]:
import pandas as pd
import numpy as np
import holidays
import lightgbm as lgb
import gc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# -----------------------------
# 1. Load dataset
# -----------------------------
df = pd.read_csv("./amazon_sales_report.csv", parse_dates=['Date'])
df = df[df['Status'].str.lower() == 'shipped']

# -----------------------------
# 2. Time-based features
# -----------------------------
df['day_of_week'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month
df['quarter'] = df['Date'].dt.quarter
df['day_of_month'] = df['Date'].dt.day
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
df['week_of_year'] = df['Date'].dt.isocalendar().week.astype(int)
df['year'] = df['Date'].dt.year

# -----------------------------
# 3. Holidays
# -----------------------------
years = df['Date'].dt.year.unique()
ind_holidays = holidays.India(years=years)
holiday_df = pd.DataFrame(list(ind_holidays.items()), columns=['Date','holiday_tag'])
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])
df = df.merge(holiday_df, on='Date', how='left')
df['holiday_tag'] = df['holiday_tag'].fillna('None')
df = pd.get_dummies(df, columns=['holiday_tag'], drop_first=False)

# -----------------------------
# 4. Promotion features
# -----------------------------
df['promotion_flag'] = df['promotion-ids'].notnull().astype(int)
df = df.sort_values(['SKU','Date'])
df['recent_promo_count_7d'] = df.groupby('SKU')['promotion_flag'].rolling(7, min_periods=1).sum().reset_index(0,drop=True)

# -----------------------------
# 5. Price / SKU features
# -----------------------------
df['amount_per_unit'] = df['Amount'] / df['Qty']
df['SKU_popularity_score'] = df.groupby('SKU')['Qty'].transform('sum')

# -----------------------------
# 6. Lag & rolling features
# -----------------------------
for lag in [1,7,30]:
    df[f'sales_t-{lag}'] = df.groupby('SKU')['Qty'].shift(lag)

df['rolling_avg_7d'] = df.groupby('SKU')['Qty'].transform(lambda x: x.rolling(7, min_periods=1).mean())
df['rolling_avg_30d'] = df.groupby('SKU')['Qty'].transform(lambda x: x.rolling(30, min_periods=1).mean())
df['sales_diff_1d'] = df['Qty'] - df['sales_t-1']
df['sales_pct_change_7d'] = (df['Qty'] - df['sales_t-7']) / df['sales_t-7']
df['sales_pct_change_7d'].replace([np.inf,-np.inf],0,inplace=True)

df['month_year'] = df['Date'].dt.to_period('M')
df['cumulative_month_sales'] = df.groupby(['SKU','month_year'])['Qty'].cumsum()

# -----------------------------
# 7. One-hot encoding for categorical features
# -----------------------------
df = pd.get_dummies(df, columns=['Sales Channel','Fulfilment','ship-service-level','Category'], drop_first=True)
df['B2B'] = df['B2B'].fillna(0).astype(int)

# -----------------------------
# 8. Geographic features
# -----------------------------
df['city_sales_avg'] = df.groupby(['SKU','ship-city'])['Qty'].transform('mean').fillna(0)
df['state_sales_avg'] = df.groupby(['SKU','ship-state'])['Qty'].transform('mean').fillna(0)

# -----------------------------
# 9. Statistical features
# -----------------------------
df['rolling_std_7d'] = df.groupby('SKU')['Qty'].transform(lambda x: x.rolling(7, min_periods=1).std())
df['sales_momentum'] = df['rolling_avg_7d'] / (df['rolling_avg_30d'] + 1e-6)

# -----------------------------
# 10. Drop unnecessary columns
# -----------------------------
drop_cols = ['Order ID','ASIN','fulfilled-by','Unnamed: 22','promotion-ids',
             'ship-city','ship-state','ship-country','ship-postal-code']

df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
df.dropna(subset=['sales_t-1','sales_t-7','sales_t-30'], inplace=True)
df['Qty'] = df['Qty'].astype(int)

# -----------------------------
# 11. Encode remaining categorical columns
# -----------------------------
cat_cols = ['Status','Style','SKU','Size','Courier Status','currency']
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

# -----------------------------
# 12. Train-test split (time-based)
# -----------------------------
train = df[df['Date'] < '2022-06-01']
test = df[df['Date'] >= '2022-06-01']


col = 'Amount'

# Compute mean ignoring zeros and NaNs
df['Amount'] = df['Amount'].fillna(df['Amount'].mean())




target = 'Amount'
features = [c for c in df.columns if c not in ['Amount','Date','month_year']]


X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

# -----------------------------
# 13. LightGBM Dataset
# -----------------------------
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols)

params = {
    'objective':'regression',
    'metric':'rmse',
    'boosting_type':'gbdt',
    'num_leaves':128,
    'learning_rate':0.05,
    'feature_fraction':0.9,
    'bagging_fraction':0.8,
    'bagging_freq':5
}

# -----------------------------
# 14. Train model
# -----------------------------
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data,test_data],
    valid_names=['train','test'],
    num_boost_round=2000,
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(100)]
)

# -----------------------------
# 15. Predict & evaluate
# -----------------------------
y_pred = model.predict(X_test)

print("First 5 predictions:", y_pred[:5])
print("First 5 predictions:", y_test[:5])

# print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
# print("MAE:", mean_absolute_error(y_test, y_pred))

# # Safe MAPE
# epsilon = 1e-6
# mape = np.mean(np.abs((y_test - y_pred)/(y_test+epsilon))) * 100
# print("MAPE:", mape)

# -----------------------------
# 16. Cleanup
# -----------------------------
del model
gc.collect()


/tmp/ipykernel_5474/1342396935.py:13: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./amazon_sales_report.csv", parse_dates=['Date'])
/tmp/ipykernel_5474/1342396935.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("./amazon_sales_report.csv", parse_dates=['Date'])
/tmp/ipykernel_5474/1342396935.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perfo

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001412 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1423
[LightGBM] [Info] Number of data points in the train set: 13147, number of used features: 40
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 639.807180
Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 18.5736	test's rmse: 26.1405
[200]	train's rmse: 14.1525	test's rmse: 24.3346
Early stopping, best iteration is:
[184]	

16